In [1]:
n_threads = 16

In [2]:
import os
if n_threads is not None:
    os.environ["OMP_NUM_THREADS"] = str(n_threads)

In [3]:
import torch
if n_threads is not None:
    torch.set_num_threads(n_threads)

In [4]:
import pandas as pd

from xopt import VOCS
from lume_model.utils import variables_from_yaml
from lume_model.torch import LUMEModule, PyTorchModel

In [5]:
model_path = "lcls_cu_injector_nn_model/"

# load sim_to_nn transformers
input_sim_to_nn = torch.load(model_path + "model/input_sim_to_nn.pt")
output_sim_to_nn = torch.load(model_path + "model/output_sim_to_nn.pt")

# load pv_to_sim transformers
input_pv_to_sim = torch.load(model_path + "model/input_pv_to_sim.pt")
output_pv_to_sim = torch.load(model_path + "model/output_pv_to_sim.pt")

# load in- and output variable specification
input_variables, output_variables = variables_from_yaml(open(model_path + "model/pv_variables.yml"))

# create LUME-model
lume_model = PyTorchModel(
    model_file=model_path + "model/model.pt",
    input_variables=input_variables,
    output_variables=output_variables,
    input_transformers=[input_pv_to_sim, input_sim_to_nn],
    output_transformers=[output_sim_to_nn, output_pv_to_sim],
)

# wrap in LUMEModule
lume_module = LUMEModule(
    model=lume_model,
    feature_order=lume_model.features,
    output_order=lume_model.outputs,
)

In [6]:
vocs = VOCS(variables = {k: v.value_range for k, v in input_variables.items()})

In [7]:
env_variable = "OMP_NUM_THREADS"
if env_variable in os.environ.keys():
    print("{}: {}".format(env_variable, os.environ[env_variable]))

print("torch num_threads: {:d}".format(torch.get_num_threads()))

OMP_NUM_THREADS: 16
torch num_threads: 16


In [8]:
%%time
batch_size = 100
for i in range(5000):
    x = torch.from_numpy(pd.DataFrame(vocs.random_inputs(batch_size)).to_numpy())
    y = lume_module(x)

CPU times: user 8.85 s, sys: 25.1 ms, total: 8.87 s
Wall time: 8.88 s
